# Pb_Multiphase_Enthalpie

In [ ]:
from trustutils import run

run.introduction("Elie Saikali & Yannick Gorsse")
run.TRUST_parameters()

In [ ]:
import pandas as pd
import os, sys
from string import Template

run.reset()
run.initBuildDirectory()

cp = 1000.0
conduc = 0.1
R = 8.31446261815324
gamma = 4.4
T = 231.44325100000003 # en C

dis = [("VDF", "", "-"), ("Polymac_P0", "polyedriser dom", "o")]
fluids = [("Incompressible", f"Fluide_Incompressible {{ rho Champ_Uniforme 1 1000 mu Champ_Uniforme 1 1e-3 lambda Champ_Uniforme 1 {conduc} Cp Champ_Uniforme 1 {cp} }}", cp * T ),
           ("Stiffened", f"StiffenedGas {{ gamma {gamma} pinf 6e8 lambda {conduc} mu 1e-3 }}", gamma * R / (gamma - 1.0) * (T + 273.15))]
cases = {"pb_multiphase" : ["energie_multiphase", "temperature"], "pb_multiphase_h" : ["energie_multiphase_enthalpie", "enthalpie"]}

for pb, (eq, ch) in cases.items():
    for d, p, _ in dis:
        for f, ff, h in fluids:
            val = T if pb == "pb_multiphase" else h
            run.addCaseFromTemplate("jdd.data", f"{d}/{f}/{pb}", {"poly" : p, "dis" : d, "fld" : ff, "pb" : pb, "eq" : eq, "champ" : ch, "val" : val})
        
run.printCases()
run.runCases()
run.tablePerf()

# Convergence

In [ ]:
from trustutils import plot
a = plot.Graph(label_size=16, title_size=24, legend_size=16, nY=2)
for i, (f, _, _) in enumerate(fluids):
    a.addPlot(i,f"Convergence {f}")
    for pb, (eq, ch) in cases.items():
        for d, _, m in dis:
            a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/{f}/{pb}/jdd.dt_ev", label=f"{d}/{pb}", marker=m)
    a.scale(yscale="log")


# Newton iterations

In [ ]:
import numpy as np
a = plot.Graph(label_size=16, title_size=24, legend_size=16, nY=2)

for i, (f, _, _) in enumerate(fluids):
    a.addPlot(i,f"Newton iterations {f}")
    for pb, (eq, ch) in cases.items():
        for d, _, m in dis:
            data = np.loadtxt(f"{run.BUILD_DIRECTORY}/{d}/{f}/{pb}/jdd.newton_evol", usecols=(0, 2))
            a.add(data[:,0], data[:,1], label=f"{d}/{pb}", marker=m)

    a.label("time","Itr Newton")


# Time step evolution (in seconds)

In [ ]:
a = plot.Graph(label_size=16, title_size=24, legend_size=16, nY=2)

for i, (f, _, _) in enumerate(fluids):
    a.addPlot(i,f"Time step {f}")
    for pb, (eq, ch) in cases.items():
        for d, _, m in dis:
            data = plot.loadText(f"{run.BUILD_DIRECTORY}/{d}/{f}/{pb}/jdd.dt_ev")
            a.add(data[0], data[1], label=f"{d}/{pb}", marker=m)
    a.label("time","dt")


# Vertical profiles 

In [ ]:
for son_name in ["P", "VL", "A", "RHOL", "TL", "HL", "MU", "LAM", "CP", "E"]:
    a = plot.Graph(title=son_name, label_size=16, title_size=24, legend_size=16, nY=2)
    for i, (f, _, _) in enumerate(fluids):
        a.addPlot(i,f"Vertical profiles {f}")
        for pb, (eq, ch) in cases.items():
            for d, _, m in dis:
                a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/{f}/{pb}/jdd_SONDE_{son_name}.son", label=f"{d}/{pb}", marker=m)